# MistralAI

```{=mdx}

:::tip
想要在本地运行Mistral的模型吗？请查看我们的 [Ollama集成](/docs/integrations/chat/ollama)。
:::

:::caution
您当前所在的页面是关于将Mistral模型用作 [文本补全模型](/docs/concepts/text_llms) 的文档。Mistral上许多流行的模型是 [聊天补全模型](/docs/concepts/chat_models)。

您可能需要查看 [这个页面](/docs/integrations/chat/mistral/)。
:::

```

[Mistral AI](https://mistral.ai/) 是一个提供其强大 [开源模型](https://docs.mistral.ai/getting-started/models/) 托管服务的平台。

这将帮助您开始使用LangChain的MistralAI补全模型（LLMs）。有关 `MistralAI` 功能和配置选项的详细文档，请参考 [API参考文档](https://api.js.langchain.com/classes/langchain_mistralai.MistralAI.html)。

## 概述
### 集成详情

| 类 | 包 | 本地 | 可序列化 | Python支持 | 包下载量 | 最新包 |
| :--- | :--- | :---: | :---: |  :---: | :---: | :---: |
| [MistralAI](https://api.js.langchain.com/classes/langchain_mistralai.MistralAI.html) | [`@langchain/mistralai`](https://www.npmjs.com/package/@langchain/mistralai) | ❌ | ✅ | ❌ | ![NPM - 下载量](https://img.shields.io/npm/dm/@langchain/mistralai?style=flat-square&label=%20&) | ![NPM - 版本](https://img.shields.io/npm/v/@langchain/mistralai?style=flat-square&label=%20&) |

## 准备工作

要访问MistralAI模型，您需要创建一个MistralAI账户，获取API密钥，并安装 `@langchain/mistralai` 集成包。

### 凭证

前往 [console.mistral.ai](https://console.mistral.ai/) 注册MistralAI并生成API密钥。完成此操作后，请设置 `MISTRAL_API_KEY` 环境变量：

```bash
export MISTRAL_API_KEY="your-api-key"
```

如果您希望自动追踪您的模型调用，也可以取消以下 [LangSmith](https://docs.smith.langchain.com/) API密钥的注释：

```bash
# export LANGSMITH_TRACING="true"
# export LANGSMITH_API_KEY="your-api-key"
```

### 安装

LangChain的MistralAI集成位于 `@langchain/mistralai` 包中：

```{=mdx}
import IntegrationInstallTooltip from "@mdx_components/integration_install_tooltip.mdx";
import Npm2Yarn from "@theme/Npm2Yarn";

<IntegrationInstallTooltip></IntegrationInstallTooltip>

<Npm2Yarn>
  @langchain/mistralai @langchain/core
</Npm2Yarn>

```

## 实例化

现在我们可以实例化我们的模型对象并生成聊天补全：

In [2]:
import { MistralAI } from "@langchain/mistralai"

const llm = new MistralAI({
  model: "codestral-latest",
  temperature: 0,
  maxTokens: undefined,
  maxRetries: 2,
  // other params...
})

## 调用

In [3]:
const inputText = "MistralAI is an AI company that "

const completion = await llm.invoke(inputText)
completion

 has developed Mistral 7B, a large language model (LLM) that is open-source and available for commercial use. Mistral 7B is a 7 billion parameter model that is trained on a diverse and high-quality dataset, and it has been fine-tuned to perform well on a variety of tasks, including text generation, question answering, and code interpretation.

MistralAI has made Mistral 7B available under a permissive license, allowing anyone to use the model for commercial purposes without having to pay any fees. This has made Mistral 7B a popular choice for businesses and organizations that want to leverage the power of large language models without incurring high costs.

Mistral 7B has been trained on a diverse and high-quality dataset, which has enabled it to perform well on a variety of tasks. It has been fine-tuned to generate coherent and contextually relevant text, and it has been shown to be capable of answering complex questions and interpreting code.

Mistral 7B is also a highly efficient mo

## 链式调用

我们可以将完成模型与提示模板[链式调用](/docs/how_to/sequence/)，如下所示：

In [4]:
import { PromptTemplate } from "@langchain/core/prompts"

const prompt = PromptTemplate.fromTemplate("How to say {input} in {output_language}:\n")

const chain = prompt.pipe(llm);
await chain.invoke(
  {
    output_language: "German",
    input: "I love programming.",
  }
)


I love programming.

Ich liebe Programmieren.

In German, the phrase "I love programming" is translated as "Ich liebe Programmieren." The word "programming" is translated to "Programmieren," and "I love" is translated to "Ich liebe."


由于Mistral LLM是一个补全模型，因此它也允许您向提示中插入一个`suffix`（后缀）。调用模型时，可以通过如下调用选项传递后缀：

In [5]:
const suffixResponse = await llm.invoke(
  "You can print 'hello world' to the console in javascript like this:\n```javascript", {
    suffix: "```"
  }
);
console.log(suffixResponse);


console.log('hello world');
```


如第一个示例所示，模型生成了请求的 `console.log('hello world')` 代码片段，但也包含了额外不需要的文本。通过添加后缀，我们可以限制模型仅将补全内容生成到该后缀为止（在此示例中为三个反引号）。这样，我们就可以通过自定义输出解析器轻松地解析补全内容，并仅提取所需的结果，而不包含后缀。

In [1]:
import { MistralAI } from "@langchain/mistralai";

const llmForFillInCompletion = new MistralAI({
  model: "codestral-latest",
  temperature: 0,
});

const suffix = "```";

const customOutputParser = (input: string) => {
  if (input.includes(suffix)) {
    return input.split(suffix)[0];
  }
  throw new Error("Input does not contain suffix.")
};

const resWithParser = await llmForFillInCompletion.invoke(
  "You can print 'hello world' to the console in javascript like this:\n```javascript", {
    suffix,
  }
);

console.log(customOutputParser(resWithParser));


console.log('hello world');



## 钩子函数

Mistral AI 支持三种事件的自定义钩子函数：beforeRequest、requestError 和 response。每种钩子类型的函数签名示例如下所示：

In [ ]:
const beforeRequestHook = (req: Request): Request | void | Promise<Request | void> => {
    // Code to run before a request is processed by Mistral
};

const requestErrorHook = (err: unknown, req: Request): void | Promise<void> => {
    // Code to run when an error occurs as Mistral is processing a request
};

const responseHook = (res: Response, req: Request): void | Promise<void> => {
    // Code to run before Mistral sends a successful response
};

将这些钩子添加到聊天模型时，可以将它们作为参数传递，它们会自动被添加：

In [ ]:
import { ChatMistralAI } from "@langchain/mistralai" 

const modelWithHooks = new ChatMistralAI({
    model: "mistral-large-latest",
    temperature: 0,
    maxRetries: 2,
    beforeRequestHooks: [ beforeRequestHook ],
    requestErrorHooks: [ requestErrorHook ],
    responseHooks: [ responseHook ],
    // other params...
});

或者在实例化后分配并手动添加它们：

In [ ]:
import { ChatMistralAI } from "@langchain/mistralai" 

const model = new ChatMistralAI({
    model: "mistral-large-latest",
    temperature: 0,
    maxRetries: 2,
    // other params...
});

model.beforeRequestHooks = [ ...model.beforeRequestHooks, beforeRequestHook ];
model.requestErrorHooks = [ ...model.requestErrorHooks, requestErrorHook ];
model.responseHooks = [ ...model.responseHooks, responseHook ];

model.addAllHooksToHttpClient();

方法 addAllHooksToHttpClient 在分配完整的更新后的钩子列表之前，会清除所有当前已添加的钩子，以避免钩子重复。

可以逐个移除钩子，也可以一次性从模型中清除所有钩子。

In [ ]:
model.removeHookFromHttpClient(beforeRequestHook);

model.removeAllHooksFromHttpClient();

## API 参考文档

有关 MistralAI 所有功能和配置的详细文档，请访问 API 参考页面: https://api.js.langchain.com/classes/langchain_mistralai.MistralAI.html